In [6]:
## Читаем с автоматическим офсетом
from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group',
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': True        # Включение автоматического коммита офсетов
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-topic'])
try:
    for _ in range(5):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()


KafkaException: KafkaError{code=UNKNOWN_TOPIC_OR_PART,val=3,str="Subscribed topic not available: my-topic: Broker: Unknown topic or partition"}

## Читаем с РУЧНЫМ офсетом

In [2]:

from confluent_kafka import Consumer, KafkaException
# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group-222222222', ## другая группа
    'auto.offset.reset': 'earliest',  # Чтение с самого начала, если офсеты не найдены
    'enable.auto.commit': False        # 
}
# Создание консюмера
consumer = Consumer(conf)
# Подписка на топик
consumer.subscribe(['my-topic'])
try:
    for _ in range(15):
        msg = consumer.poll(timeout=1.0)  # Ожидание новых сообщений  
        if msg is None:
            continue
        if msg.error():
            raise KafkaException(msg.error())
        print(f'Received message: {msg.value().decode("utf-8")}')
        
finally:
    # Закрытие консюмера
    consumer.close()

Received message: qwe
Received message: asd
Received message: zxc
Received message: qwf
Received message: asd
Received message: hello
Received message: gr1
Received message: asd
Received message: asd
Received message: asdzxv
Received message: 123
Received message: 123
Received message: 123
Received message: я1
Received message: Hello Kafka 0


Чтение офсетов для конкретной группы консумеров
Для чтения текущих офсетов для группы консумеров можно использовать метод committed(partitions). Этот метод возвращает список офсетов для указанных разделов (partitions).

In [3]:
from confluent_kafka import Consumer, TopicPartition

# Конфигурация консюмера
conf = {
    'bootstrap.servers': 'localhost:9092',
    'group.id': 'my-group',
    'auto.offset.reset': 'earliest',
    'enable.auto.commit': False
}

consumer = Consumer(conf)
consumer.subscribe(['my-topic'])

# Получение текущих офсетов для группы консумеров по разделам топика
partitions = [TopicPartition('my-topic', partition=0)]
offsets = consumer.committed(partitions)

for tp in offsets:
    print(f'Partition: {tp.partition}, Offset: {tp.offset}')

consumer.close()


Partition: 0, Offset: 24


### Producer


In [5]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer'                # Идентификатор клиента
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Отправка сообщений
try:
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Ожидание отправки всех сообщений
    producer.flush()

except Exception as e:
    print(f'Error while producing: {e}')

finally:
    # Закрытие продюсера
    print('finally')


Message delivered to my-topic [0] at offset 39
Message delivered to my-topic [0] at offset 40
Message delivered to my-topic [0] at offset 41
Message delivered to my-topic [0] at offset 42
finally


### exactly-once semantics, EOS

 idempotent producer и настроек для транзакций
 Конфигурация продюсера:

enable.idempotence: True: Включает идемпотентный режим для продюсера.
transactional.id: Уникальный идентификатор для транзакций. Он должен быть уникальным для каждого продюсера в вашем приложении.

In [24]:
from confluent_kafka import Producer

# Функция для обработки успешной доставки сообщений
def delivery_report(err, msg):
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}')

# Конфигурация продюсера
conf = {
    'bootstrap.servers': 'localhost:9092',  # Адрес Kafka брокера
    'client.id': 'my-producer',              # Идентификатор клиента
    'enable.idempotence': True,               # Включение идемпотентного режима
    'acks': 'all',                            # Подтверждение от всех реплик
    'retries': 5,                             # Количество попыток повторной отправки
    'transactional.id': 'my-transactional-id'  # Идентификатор транзакции
}

# Создание экземпляра продюсера
producer = Producer(conf)

# Инициализация транзакции
producer.init_transactions()

try:
    # Начало транзакции
    producer.begin_transaction()

    # Отправка сообщений
    for i in range(4):
        key = f'key-{i}'                # Ключ сообщения (опционально)
        value = f'Hello Kafka {i}'      # Значение сообщения
        producer.produce('my-topic', key=key, value=value, callback=delivery_report)

        # Обработка асинхронных операций
        producer.poll(0)  # Проверка на наличие ошибок и вызов обратных функций

    # Завершение транзакции
    producer.commit_transaction()

except Exception as e:
    print(f'Error while producing: {e}')
    # Отмена транзакции в случае ошибки
    producer.abort_transaction()

finally:
    # Ожидание отправки всех сообщений
    producer.flush()


%4|1729585021.552|GETPID|my-producer#producer-22| [thrd:main]: Failed to acquire transactional PID from broker TxnCoordinator/1: Broker: Not coordinator: retrying


Message delivered to my-topic [0] at offset 34
Message delivered to my-topic [0] at offset 35
Message delivered to my-topic [0] at offset 36
Message delivered to my-topic [0] at offset 37


## Schema registry

In [11]:
import requests
import json

In [39]:

response = requests.get("http://localhost:8081/subjects")
response.json()

['my-avro-topic-value', 'my-topic-value']

Регистрация новой схемы

In [28]:
import requests
import json

url = 'http://localhost:8081/subjects/my-topic-value/versions'
schema = {
    "schema": json.dumps({
        "type": "record",
        "name": "User",
        "fields": [
            {"name": "name", "type": "string"},
            {"name": "age", "type": "int"}
        ]
    })
}

response = requests.post(url, json=schema)

if response.status_code == 200:
    schema_id = response.json().get('id')
    print('Схема зарегистрирована с ID:', schema_id)
else:
    print('Ошибка при регистрации схемы:', response.text)


Схема зарегистрирована с ID: 1


In [21]:
schema_id = 1  # Замените на нужный ID
url = f'http://localhost:8081/schemas/ids/{schema_id}'
response = requests.get(url)

if response.status_code == 200:
    schema = response.json().get('schema')
    print('Полученная схема:', schema)
else:
    print('Ошибка при получении схемы:', response.text)


Полученная схема: {"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"age","type":"int"}]}


AVRO

In [18]:
response = requests.get("http://localhost:8081/subjects/my-topic-value/versions/latest")
response.json()

{'subject': 'my-topic-value',
 'version': 1,
 'id': 1,
 'schema': '{"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"age","type":"int"}]}'}

In [36]:
class User(object):
    def __init__(self, name, address, favorite_number, favorite_color):
        self.name = name
        self.favorite_number = favorite_number
        self.favorite_color = favorite_color
        # address should not be serialized, see user_to_dict()
        self._address = address

In [37]:
def user_to_dict(user, ctx):
    # User._address must not be serialized; omit from dict
    return dict(name=user.name,
                favorite_number=user.favorite_number,
                favorite_color=user.favorite_color)

In [38]:
def delivery_report(err, msg):
 
    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [35]:
import os
from uuid import uuid4

from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer

topic = "my-topic"
schema = "user.avsc"

#path = os.path.realpath(os.path.dirname(__file__))
with open(f"{schema}") as f:
    schema_str = f.read()

schema_registry_conf = {'url': "http://localhost:8081/"}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

avro_serializer = AvroSerializer(schema_registry_client,
                                    schema_str,
                                    user_to_dict)

string_serializer = StringSerializer('utf_8')

producer_conf = {'bootstrap.servers': 'localhost:9092'}

producer = Producer(producer_conf)

print("Producing user records to topic {}. ^C to exit.".format(topic))
while True:
    # Serve on_delivery callbacks from previous calls to produce()
    producer.poll(0.0)
    try:
        user_name = input("Enter name: ")
        user_address = input("Enter address: ")
        user_favorite_number = int(input("Enter favorite number: "))
        user_favorite_color = input("Enter favorite color: ")
        user = User(name=user_name,
                    address=user_address,
                    favorite_color=user_favorite_color,
                    favorite_number=user_favorite_number)
        producer.produce(topic=topic,
                            key=string_serializer(str(uuid4())),
                            value=avro_serializer(user, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
    except KeyboardInterrupt:
        break
    except ValueError:
        print("Invalid input, discarding record...")
        continue

print("\nFlushing records...")
producer.flush()

Producing user records to topic my-topic. ^C to exit.


SchemaRegistryError: Schema being registered is incompatible with an earlier schema for subject "my-topic-value", details: [{errorType:'READER_FIELD_MISSING_DEFAULT_VALUE', description:'The field 'favorite_number' at path '/fields/1' in the new schema has no default value and is missing in the old schema', additionalInfo:'favorite_number'}, {errorType:'READER_FIELD_MISSING_DEFAULT_VALUE', description:'The field 'favorite_color' at path '/fields/2' in the new schema has no default value and is missing in the old schema', additionalInfo:'favorite_color'}, {oldSchemaVersion: 1}, {oldSchema: '{"type":"record","name":"User","fields":[{"name":"name","type":"string"},{"name":"age","type":"int"}]}'}, {compatibility: 'BACKWARD'}] io.confluent.kafka.schemaregistry.rest.exceptions.RestIncompatibleSchemaException: Schema being registered is incompatible with an earlier schema for subject "my-topic-value", details: [{errorType:'READER_FIELD_MISSING_DEFAULT_VALUE', description:'The field 'favorite_n

### KAFKA CONNECT

In [4]:
import requests
import json

response = requests.get("http://localhost:8083/connectors")
response.json()

[]

In [5]:
response = requests.get("http://localhost:8083/connectors/my_table-connector")
response.json()

{'error_code': 404, 'message': 'Connector my_table-connector not found'}

Создаем коннектор для чтения из файла

## DEBEZIUM CDC

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
import os
import time

pwd = "secret"
uid = "debezium"
server = "localhost"
db = "mydb"
port = "5432"
#
engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:{port}/{db}')

In [55]:
df = pd.read_sql('select * from public.my_table', engine)
df

,id,name
0,1,John Doe
1,2,Alice
2,3,Bob
3,4,John Doe
4,5,Alice
5,6,Bob


In [74]:
# Создайте DataFrame с данными для вставки
data = {'name': ['asd----', 'Alice', 'Bob']}
df = pd.DataFrame(data)

# Вставка данных в my_table
df.to_sql('my_table', engine, if_exists='append', index=False)

3

In [57]:
response = requests.get("http://localhost:8083/")
response.json()

{'version': '2.6.1',
 'commit': '6b2021cd52659cef',
 'kafka_cluster_id': 'MkU3OEVBNTcwNTJENDM2Qk'}

In [69]:
postgres_connector = {
    "name": "my_table-connector",
    "config": {
        "connector.class": "io.debezium.connector.postgresql.PostgresConnector",
        "database.hostname": "postgr",  
        "database.port": "5432",
        "database.user": "debezium",      
        "database.password": "secret", 
        "database.dbname": "mydb",  
        "plugin.name": "pgoutput",
        "database.server.name": "source",
        "key.converter.schemas.enable": "false",
        "value.converter.schemas.enable": "false",
        "transforms": "unwrap",
        "transforms.unwrap.type": "io.debezium.transforms.ExtractNewRecordState",
        "value.converter": "org.apache.kafka.connect.json.JsonConverter",
        "key.converter": "org.apache.kafka.connect.json.JsonConverter",
        "table.include.list": "public.my_table", 
        "slot.name": "dbz_sales_transaction_slot"
    }
}


In [70]:
response = requests.get('http://localhost:8083/connectors/')
response.json()

[]

In [71]:
url = 'http://localhost:8083/connectors/'


# Отправляем данные как JSON
response = requests.post(url, json = postgres_connector)

# Проверяем статус код ответа
if response.status_code == 200:
    print("Данные успешно отправлены.")
    print("Ответ сервера:", response.json())  # Если сервер возвращает JSON-ответ
else:
    print("Ошибка при отправке данных:", response.status_code, response.text)

Ошибка при отправке данных: 201 {"name":"my_table-connector","config":{"connector.class":"io.debezium.connector.postgresql.PostgresConnector","database.hostname":"postgr","database.port":"5432","database.user":"debezium","database.password":"secret","database.dbname":"mydb","plugin.name":"pgoutput","database.server.name":"source","key.converter.schemas.enable":"false","value.converter.schemas.enable":"false","transforms":"unwrap","transforms.unwrap.type":"io.debezium.transforms.ExtractNewRecordState","value.converter":"org.apache.kafka.connect.json.JsonConverter","key.converter":"org.apache.kafka.connect.json.JsonConverter","table.include.list":"public.my_table","slot.name":"dbz_sales_transaction_slot","name":"my_table-connector"},"tasks":[],"type":"source"}


In [72]:
response = requests.get('http://localhost:8083/connectors/my_table-connector')
response.json()

{'name': 'my_table-connector',
 'config': {'connector.class': 'io.debezium.connector.postgresql.PostgresConnector',
  'database.user': 'debezium',
  'database.dbname': 'mydb',
  'slot.name': 'dbz_sales_transaction_slot',
  'transforms': 'unwrap',
  'database.server.name': 'source',
  'database.port': '5432',
  'plugin.name': 'pgoutput',
  'key.converter.schemas.enable': 'false',
  'database.hostname': 'postgr',
  'database.password': 'secret',
  'value.converter.schemas.enable': 'false',
  'name': 'my_table-connector',
  'transforms.unwrap.type': 'io.debezium.transforms.ExtractNewRecordState',
  'value.converter': 'org.apache.kafka.connect.json.JsonConverter',
  'table.include.list': 'public.my_table',
  'key.converter': 'org.apache.kafka.connect.json.JsonConverter'},
 'tasks': [{'connector': 'my_table-connector', 'task': 0}],
 'type': 'source'}

In [73]:
response = requests.get('http://localhost:8083/connectors/my_table-connector/status')
response.json()

{'name': 'my_table-connector',
 'connector': {'state': 'RUNNING', 'worker_id': '172.18.0.5:8083'},
 'tasks': [{'id': 0, 'state': 'RUNNING', 'worker_id': '172.18.0.5:8083'}],
 'type': 'source'}

In [2]:
!pip show confluent-kafka

Name: confluent-kafka
Version: 2.6.0
Summary: Confluent's Python client for Apache Kafka
Home-page: https://github.com/confluentinc/confluent-kafka-python
Author: Confluent Inc
Author-email: support@confluent.io
License: 
Location: /root/myenv/lib/python3.12/site-packages
Requires: 
Required-by: 


In [64]:
from confluent_kafka.admin import AdminClient   

# Определите адреса брокеров Kafka
bootstrap_servers = ['localhost:9092']

# Настройка Kafka
config = {
    'bootstrap.servers': 'localhost:9092'  # Адрес вашего Kafka-брокера
}
# Создайте экземпляр KafkaAdminClient
admin_client = AdminClient(config)

# Запрос метаданных для получения списка топиков
metadata = admin_client.list_topics(timeout=10)

# Получение списка топиков
topics = metadata.topics

# Вывод списка топиков
for topic in topics:
    print(topic)

connect_status
connect_offsets
connect_configs
_schemas
source.public.my_table
__consumer_offsets


In [ ]:
from kafka import KafkaConsumer
bootstrap_servers = ['localhost:29092']
consumer = KafkaConsumer( bootstrap_servers=bootstrap_servers)
consumer.topics()

In [ ]:
import json
topicName = 'source.public.my_table'
# Initialize consumer variable
consumer = KafkaConsumer (topicName , auto_offset_reset='earliest', 
                          bootstrap_servers = bootstrap_servers, group_id='sales-transactions')

# Read and print message from consumer
for msg in consumer:
    print(json.loads(msg.value))

In [77]:
# Удаление
#response = requests.delete('http://localhost:8083/connectors/my_table-connector')
#response.json()

Received message: я1


Received message: qwe
Received message: asd
Received message: zxc
Received message: qwf
Received message: asd
Received message: hello
Received message: gr1
Received message: asd
Received message: asd
Received message: asdzxv
Received message: 123
Received message: 123
Received message: 123
Received message: я1


Partition: 0, Offset: 14
